# NOTE
# This notebook server was started with command below, in which case "sc" is a spark context 

IPYTHON_OPTS="notebook --certfile=~/cert/mycert.pem --keyfile ~/cert/mykey.key" $SPARK_HOME/bin/pyspark --master spark://spark1:7077 --jars $SPARK_HOME/jars/elasticsearch-hadoop-2.2.0.jar


In [1]:
import os
print os.getcwd()

/root/notebooks


In [4]:
#lat/lon grid class
import sys
sys.path.append('../Infrastructure_Capstone')
import os
import math
from shapely.geometry import Polygon
from pyproj import Proj
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch.client import indices
#from dataStorage import upload_to_Elasticsearch
import ConfigParser
from pprint import pprint
from copy import deepcopy
import datetime as dt
from dateutil.parser import parse

#read in the config file
#os.chdir('~/Infrastructure_Capstone')
config = ConfigParser.ConfigParser()
config.read('../Infrastructure_Capstone/config/capstone_config.ini')

ES_url = config.get('ElasticSearch','host')
ES_password = config.get('ElasticSearch','password')
ES_username= config.get('ElasticSearch','username')


In [5]:
#print sc to see what it is
print sc

In [6]:
grid_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable", 
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
    conf={ "es.resource" : "dataframe/rows", "es.nodes" : ES_url, 
          "es.net.http.auth.user" : ES_username, 
          "es.net.http.auth.pass" : ES_password })

grid_rdd.first()

(u'2013-02-06T13:00:00_10306',
 {u'grid_day': 6,
  u'grid_dayOfWeek': 3,
  u'grid_fullDate': u'2013-02-06T12:00:00-06:00',
  u'grid_hourOfDay': 13,
  u'grid_id': u'2013-02-06T13:00:00_10306',
  u'grid_isAccident': 0,
  u'grid_month': 2,
  u'grid_zipcode': 10306})

In [9]:
weather_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable", 
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
    conf={ "es.resource" : "weather_observations/weather_observations", "es.nodes" : ES_url, 
          "es.net.http.auth.user" : ES_username, 
          "es.net.http.auth.pass" : ES_password })

In [10]:
#print first row in accidents
print weather_rdd.first()

(u'00178_20160208_11', {u'AltimeterFlag': None, u'ValueForWindCharacterFlag': None, u'DewPointCelsius': -3.0, u'DateTime': u'2016-02-08T11:55:00', u'WindSpeed': 9.0, u'WetBulbFarenheit': 32.0, u'WetBulbFarenheitFlag': None, u'WeatherType': None, u'DewPointCelsiusFlag': None, u'PressureTendencyFlag': None, u'WetBulbCelsiusFlag': None, u'SeaLevelPressure': u'99999.0', u'RecordType': u'AA', u'SkyConditionFlag': None, u'VisibilityFlag': None, u'Visibility': 10.0, u'obs_id': u'00178_20160208_11', u'DewPointFarenheit': 27.0, u'Date': u'2016-02-08T00:00:00', u'Altimeter': 29.8, u'WetBulbCelsius': -0.0, u'SkyCondition': u'OVC060', u'RelativeHumidity': 73.0, u'DryBulbFarenheitFlag': None, u'DewPointFarenheitFlag': None, u'DryBulbFarenheit': 35.0, u'PressureChangeFlag': None, u'DryBulbCelsius': 1.7, u'DryBulbCelsiusFlag': None, u'Time': u'1900-01-01T11:55:00', u'StationPressure': 29.78, u'RelativeHumidityFlag': None, u'RecordTypeFlag': None, u'WindSpeedFlag': None, u'WeatherTypeFlag': None, u'Se

#Grid Development

In [46]:
#Lambert Conformal Conic projection centered on the polygon definition
lcc = '+proj=lcc +lat_1=%s +lat_2=%s +lat_0=%s +lon_0=%s +preserve_units = True +ellps=clrk66' % (33,45,40.6795535,-74.0006865)
with open('./Infrastructure_Capstone/flatDataFiles/NYC_polygon.json','r') as geo_file:
    geo = geojson.load(geo_file)
    geo_poly = geo['features'][0]
    poly = list(geojson.utils.coords(geo_poly))
    #print len(poly)

    
##LET'S START WITH JUST A FEW DAYS
data_start = dt.datetime(2012,6,30,17,0)
data_end = dt.datetime(2012,7,3,17,0)
#data_end = dt.datetime(2016,2,5,16,0)
time_next = data_start

times = []
#create a list of time range starts to iterate over
while time_next <= data_end:
    times.append(time_next)
    time_next+=dt.timedelta(seconds=86400)
print len(times)
data_bounds = [-74.25909,40.477399,-73.700009,40.917577]

##DEFINE THE GRID OBJECT, FILTER FOR NYC ONLY
grid = SearchGrid(data_bounds,50,filter_poly=poly,proj_str=lcc)
from pprint import pprint
pprint(grid.temporal_grid().next())

4
<type 'str'>
{'grid_boundary': {'coordinates': [[[-74.25909, 40.477399],
                                    [-74.24775366120572,
                                     40.477572560152176],
                                    [-74.24798024634923,
                                     40.486229302288606],
                                    [-74.2593180408524, 40.48605572013411],
                                    [-74.25909, 40.477399]]],
                   'type': 'polygon'},
 'grid_center': [-74.25353548711237, 40.48181428632108],
 'grid_id': '0_0_19700101_0000',
 'grid_time': datetime.datetime(1970, 1, 1, 0, 0)}


In [55]:
from copy import deepcopy
import json
def jsonify(grid):
    #convert json-type object to json string
    new_grid = deepcopy(grid)
    new_grid['grid_time'] = int((new_grid['grid_time'] - dt.datetime(1970,1,1,0,0,0)).total_seconds()*1000) #convert datetime to milliseconds since 1970
    return json.dumps(new_grid)
    #return new_grid
    
def pyspark_upload_grid_doc_to_es(grid,start,end):
    #input: a SearchGrid, start and end times
    #uploads an rdd of the grid to ES
    import json
    grid = grid.temporal_grid(start,end)
    
    grid_rdd = sc.parallelize(grid).map(lambda doc: (doc['grid_id'],jsonify(doc)))
    #print grid_rdd.count()
    #print grid_rdd.first()
    grid_rdd.saveAsNewAPIHadoopFile(
        path='-', 
        outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
        keyClass="org.apache.hadoop.io.NullWritable", 
        valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
        conf={"es.resource" : "nyc_grid/grid", 
              "es.input.json": "true",
              "es.nodes" : ES_url, 
              "es.net.http.auth.user" : ES_username, 
              "es.net.http.auth.pass" : ES_password })
    

In [56]:
#THIS KEEPS THROWING EXCEPTIONS -- NEED TO WORK ON IT
for time in times:
    pyspark_upload_grid_doc_to_es(grid,time,time+dt.timedelta(seconds=86400))
    
print "Done!"

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.saveAsNewAPIHadoopFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 46.0 failed 1 times, most recent failure: Lost task 1.0 in stage 46.0 (TID 113, localhost): org.elasticsearch.hadoop.rest.EsHadoopNoNodesLeftException: Connection error (check network and/or proxy settings)- all nodes failed; tried [[169.53.138.85:9200]] 
	at org.elasticsearch.hadoop.rest.NetworkClient.execute(NetworkClient.java:142)
	at org.elasticsearch.hadoop.rest.RestClient.execute(RestClient.java:423)
	at org.elasticsearch.hadoop.rest.RestClient.execute(RestClient.java:415)
	at org.elasticsearch.hadoop.rest.RestClient.bulk(RestClient.java:145)
	at org.elasticsearch.hadoop.rest.RestRepository.tryFlush(RestRepository.java:225)
	at org.elasticsearch.hadoop.rest.RestRepository.flush(RestRepository.java:248)
	at org.elasticsearch.hadoop.rest.RestRepository.doWriteToIndex(RestRepository.java:20